[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA3/blob/main/lab7/lab7.ipynb)

# Práctica 7: Autoaprendizaje

## Pre-requisitos

### Instalar paquetes

Si la práctica requiere algún paquete de Python, habrá que incluir una celda en la que se instalen. Si usamos un paquete que se ha utilizado en prácticas anteriores, podríamos dar por supuesto que está instalado pero no cuesta nada satisfacer todas las dependencias en la propia práctica para reducir las dependencias entre ellas.

### NOTA: En <font color='red'>Google Colab</font> hay que instalar los paquetes EN CADA EJECUCIÓN

In [ ]:
# Ejemplo de instalación de tensorflow 2.0
#%tensorflow_version 2.x
# !pip3 install tensorflow  # NECESARIO SOLO SI SE EJECUTA EN LOCAL
import tensorflow as tf

# Hacemos los imports que sean necesarios
import numpy as np

# Autoaprendizaje sobre Fashion-MNIST

Lo primero que tenemos que hacer es cargar el dataset. Tomaremos el 5% de los datos como etiquetados (`x_train` e `y_train`) y el resto como no etiquetados (`unlabeled_train`).

In [ ]:
labeled_data = 0.05 # Vamos a usar el etiquetado de sólo el 5% de los datos
np.random.seed(42)

(x_train, y_train), (x_test, y_test), = tf.keras.datasets.fashion_mnist.load_data()

indexes = np.arange(len(x_train))
np.random.shuffle(indexes)
ntrain_data = int(labeled_data*len(x_train))
unlabeled_train = x_train[indexes[ntrain_data:]]
x_train = x_train[indexes[:ntrain_data]]
y_train = y_train[indexes[:ntrain_data]]

In [ ]:
# TODO: Haz el preprocesado que necesites aquí (Vuelve sobre esta celda tras estudiar las posteriores)
# Asegúrate de tener los shapes apropiados y de que las variables se representen de una manera que el modelo pueda predecirlas.
None

## Función de autoaprendizaje

Vamos a crear nuestra propia función de autoaprendizaje. La idea es comenzar aprendiendo un modelo para los datos etiquetados y predecir con ese modelo los datos sin etiquetar. De dichas predicciones, tomaremos aquellas en las que el modelo tiene mayor confianza (cuya probabilidad exceda un umbral `thresh`) y las añadiremos al conjunto de datos etiquetados. Repetiremos el proceso un número predeterminado de veces `train_epochs`.

El pseudocódigo es el siguiente:



**self_training** *(model, x_train, y_train, unlabeled_data, thresh, train_epochs)*

1. $train\_data, train\_label \leftarrow x\_train, y\_train$
1. **Desde** $n = 1 .. train\_epochs$ **hacer**
    1. Instancia un nuevo *model* y entrénalo usando las variables *train\_data* y *train\_label* 
    2. $y\_pred \leftarrow model(unlabeled\_data)$
    3. $y\_class, y\_value \leftarrow $ Clase ganadora en *y_pred* con su valor
    4. $train\_data, train\_label \leftarrow x\_train, y\_train$
    5. **Para cada elemento** (x_u, y_c, y_v) **de la tupla** (unlabeled_data, y_class, y_value) 
        1. **Si** $y\_v > thresh$ **entonces**
            1. Añadimos $x\_u$ e $y\_c$ a train\_data y train\_label, respectivamente.
4. Devolvemos el modelo


La función asume que `model_creator` es una función sin parámetros que instancia un modelo de Sklearn que, por tanto, cuenta con las funciones [fit](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.fit), [predict](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict), [predict_proba](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict_proba) y [score](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.score). Consulta la documentación para identificar cuál te es útil en cada momento.

In [ ]:
# TODO: Implementa el algoritmo self_training tal y como viene en el pseudocódigo.
# TODO: Imprime en cada epoch la precisión y el número de elementos en el conjunto etiquetado,

def self_training(model_creator, x_train, y_train, unlabeled_data, x_test, y_test, thresh=0.5, train_epochs=3):
    train_data = x_train.copy() # Copiamos los conjuntos de datos para no modificar los originales
    train_label = y_train.copy()
    for i in range(train_epochs):
        model = model_creator()
        #TODO Completa el algoritmo siguiendo el pseudocódigo
        None
    return model

### Entrenamos nuestro clasificador

Usa lo hecho anteriormente para entrenar tu clasificador de una manera semi-supervisada. Utiliza para ello el [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) de sklearn (vigila el parámetro probability).

In [ ]:
# Define la función para llamar al SVM
from sklearn.svm import SVC
model_func = lambda: SVC(kernel='linear', probability=True, max_iter=200)

In [1]:
# TODO: Entrena tu clasificador


## Mejorando el código

Tal como hemos visto en clase de teoría, este código puede ser mejorado de varias maneras:

  1. **Asignar más peso a las variables etiquetadas**.
  1. **Asignar un peso en función de la certeza de la predicción en las variables sin etiquetar**.

### TRABAJO: Modifica la función self_training para tener en cuenta todos los puntos mencionados anteriormente

In [ ]:
# TODO: reescribe la función self_training para incorporar las mejoras mencionadas anteriormente

def self_training_v2(model_creator, x_train, y_train, unlabeled_data, x_test, y_test, thresh=0.8, train_epochs=3):
    None


In [ ]:
# TODO: Entrena tu clasificador
None

### Creamos nuestro clasificador en Tensorflow

A continuación crearemos nuestro propio modelo en TensorFlow pero asegurándonos que tiene los métodos de la API de `sklearn` que se utilizan en `self_training`. Ya deberías de ser capaz de realizar este trabajo con muy poca ayuda. El diseño del clasificador es libre (capas densas, convolucionales, ...), puedes crearlo como quieras. Lo único que tenemos que tener en cuenta es que tenemos que encapsular nuestro modelo en una clase cuyas funciones tengan el mismo nombre (y los mismos parámetros) que las existentes en sklearn.

In [ ]:
# TODO: crea tu propio clasificador

class MiClasificador:

    def __init__(self, optimizer):
        # TODO : define el modelo
        self.model = None
        # TODO: crea el optimizador
        None
        # TODO: compila el modelo
        self.model.compile(
            loss=None,
            optimizer=optimizer,
            metrics=['accuracy']
        )
    
    def fit(self, X, y, sample_weight=None):
        # TODO: entrena el modelo. Escoge el tamaño de batch y el número de epochs que quieras
        pass

    def predict(self, X):
        # TODO: devuelve la clase ganadora
        pass
    
    def predict_proba(self, X):
        # TODO: devuelve las probabilidades de cada clase
        pass
    
    def score(self, X, y):
        # TODO: devuelve el accuracy del clasificador
        pass

    def __del__(self):
        del self.model
        tf.keras.backend.clear_session() # Necesario para liberar la memoria en GPU

### Entrenando el modelo

Crea una función que nos permita crear el modelo en cada iteración.

In [ ]:
# TODO: Entrena el modelo
None

# ¡ENHORABUENA! Has completado la práctica de auto-aprendizaje.
